# Setup

In [1]:
import nidaqmx 
import atiiaftt as ati
from nidaqmx.stream_readers import AnalogMultiChannelReader
import time as Time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ftsensor
import datetime as dt
import csv
import serial
from time import sleep

sensor = ftsensor.ftsensor(tool_offset=[0,0,0,0,0,0])

ENTER = bytes([0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFC])
EXIT = bytes([0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFD])
ZERO = bytes([0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFE])


# AK70-10 24V Motor Limit
P_MIN = -12.5
P_MAX = 12.5
V_MIN = -50  
V_MAX = 50
T_MIN = -25 
T_MAX = 25
KP_MIN = 0
KP_MAX = 500
KD_MIN = 0
KD_MAX = 5

def unpack(data):
    id = data[0]
    pos = ((data[1]*(P_MAX-P_MIN))/(pow(2,16)-1)) + P_MIN
    vel = ((data[2]*(V_MAX-V_MIN))/(pow(2,12)-1)) + V_MIN
    tor = ((data[3]*(T_MAX-T_MIN))/(pow(2,12)-1)) + T_MIN
    return id, pos, vel, tor

def receive():
    global dT, q1_old, q2_old
    # Read the response from the serial port
    received = serD.readline().decode().split()      

    # Print the received bytes
    # print("Received:", received)

    try:
        test = int(received[0])
        response = [int(x) for x in received]
        [id, p, v, t] = unpack(response)

        if id == 1:
            p = -round(p,3)
            v = -round(v,3)
            t = t
        
        else:
            p = round(p,3)
            v = round(v,3)
            t = -t

        # print the response from the Arduino
        # print([id, p, v, t])

    except (IndexError, ValueError):
        id = None
        p = 0
        v = 0
        t = 0
    return id, p, v, t

def pack_cmd(p_des, v_des, t_ff, kp, kd):
    bufs = []
    def float_to_uint(val, val_min, val_max, bits):
        val_norm = (val - val_min) / (val_max - val_min)
        val_int = int(round(val_norm * ((1 << bits) - 1)))
        return val_int
    
    p_int = float_to_uint(p_des, P_MIN, P_MAX, 16)
    v_int = float_to_uint(v_des, V_MIN, V_MAX, 12)
    t_int = float_to_uint(t_ff, T_MIN, T_MAX, 12)
    kp_int = float_to_uint(kp, KP_MIN, KP_MAX, 12)
    kd_int = float_to_uint(kd, KD_MIN, KD_MAX, 12)
    
    bufs.append((p_int >> 8) & 0xFF)
    bufs.append(p_int & 0xFF)
    bufs.append((v_int >> 4) & 0xFF)
    bufs.append(((v_int & 0xF) << 4) | ((kp_int >> 8) & 0xF))
    bufs.append(kp_int & 0xFF)
    bufs.append((kd_int >> 4) & 0xFF)
    bufs.append(((kd_int & 0xF) << 4) | ((t_int >> 8) & 0xF))
    bufs.append(t_int & 0xFF)

    return bufs

Construct Serial Connection

In [76]:
# open the serial port
serD = serial.Serial('COM4', 115200, timeout=1) #Device
serS = serial.Serial('COM9', 115200, timeout=1) #Stepper


# Set Zero

In [3]:
print("Begining")


q1_old = 0
pos2_old = 0
dT = 0

#M1
print("Starting Zero Position M1")
serD.write(bytes([0x01])+ZERO)
sleep(3)
[id, p, v, t] = receive()
p = 5
while abs(p) >= 0.001:
    serD.write(bytes([0x01])+ZERO)
    sleep(3)
    [id, p, v, t] = receive()



#M2
print("Starting Zero Position M2")
serD.write(bytes([0x02])+ZERO)
sleep(3)
[id, p, v, t] = receive()
p = 5
while abs(p) >= 0.001:
    serD.write(bytes([0x02])+ZERO)
    sleep(3)
    [id, p, v, t] = receive()

print("Finishing Zero Position")


Begining
Starting Zero Position M1
Starting Zero Position M2
Finishing Zero Position


# Test Motor

In [92]:
l1 = 0.275
l2 = 0.24
dT = 0

command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)

#M1
serD.write(bytes([0x01])+command1)
[id, p, v, t] = receive()

if id == 1:
    q1 = p

elif id == 2:
    q2 = p


#M2
serD.write(bytes([0x02])+command2)
[id, p, v, t] = receive()

if id == 1:
    q1 = p


elif id == 2:
    q2 = p



# print("q1: ",q1, "\tq2: ", q2)

x = l1*np.sin(q1) + l2*np.cos(q2)
y = l1*np.cos(q1) - l2*np.sin(q2)


print("x: ",x, "\ty: ", y)


x:  0.25166900055990765 	y:  0.1742840859793484


# Test Stepper

Note
Distance of Path = 0.09623
Duration = 13.5s

In [13]:
print("Set Zero Stepper")

FB = serS.read_all()

while True:    
    FB = serS.readline().decode('utf-8').strip()
    if FB:
        if int(FB) == 4:
            serS.write(b'2')
            break
    else:
        serS.write(b'1')



print("Ending Zero Stepper")

sleep(2)

end = 0
count = 0
step_distance = 3 #cm
step_time_stepper = step_distance*1.4
total_dis = 9.623
total_count = round(total_dis//step_distance)
while count < total_count+1:
    count +=1 
    FB = serS.readline().decode('utf-8').strip()
    if FB:
        if int(FB) == 5:
            serS.write(b'1')
            sleep(0.5)  
            serS.write(b'2')
            end = 1
            break
    
    if end == 1:
        break


    command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
    command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)

    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    #M1
    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    if id == 1:
        q1 = p
    
    elif id == 2:
        q2 = p

    #M2
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()
    if id == 1:
        q1 = p
    
    elif id == 2:
        q2 = p

    y = l1*np.cos(q1) - l2*np.sin(q2)

    
   
    serS.write(b'3')
    sleep(step_time_stepper)
    serS.write(b'2')

    


    

    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    #M1
    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    if id == 1:
        q1 = p
    
    elif id == 2:
        q2 = p

    #M2
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()
    if id == 1:
        q1 = p
    
    elif id == 2:
        q2 = p

    yn = l1*np.cos(q1) - l2*np.sin(q2)

    diff = yn-y
    
    print("y:", y, " yn: ", yn, " diff: ", diff)
        

    sleep(2)


Set Zero Stepper
Ending Zero Stepper
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0
y: 0.275  yn:  0.275  diff:  0.0


# Enter Motor and Re-bias Force Sensor

In [93]:
sensor.stop_task()

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\nidaqmx\task.py:456: ResourceWarning: Attempted to close NI-DAQmx task of name "_unnamedTask<32>" but task was already closed.
  warnings.warn(


In [94]:
##Motor

dT = 0

command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)

#M1
serD.write(bytes([0x01])+command1)
[id, p, v, t] = receive()

#M2
serD.write(bytes([0x02])+command2)
[id, p, v, t] = receive()


#M1
print("Starting Enter Motors")
serD.write(bytes([0x01])+ENTER)
[id, p, v, t] = receive()
if id == 1:
    q1 = p
    vel1 = v

elif id == 2:
    q2 = p
    vel2 = v

#M2
serD.write(bytes([0x02])+ENTER)
[id, p, v, t] = receive()

if id == 1:
    q1 = p
    vel1 = v

elif id == 2:
    q2 = p
    vel2 = v

q1_old = q1
q2_old = q2

sleep(3)

##Sensor
sensor.start_task()
current_time = Time.time()
duration = 1
values = []
while Time.time()< current_time + duration:
    values.append(np.array(sensor.read_avg_voltage()))

values = np.array(values)
values = [ft.mean() for ft in values.transpose()]
sensor.set_bias(values)
sensor.stop_task()
print(values)


Starting Enter Motors
[-0.12336207390369053, -0.08656107773806912, -0.13671236238658593, -0.05789237269978997, 0.20879452898818912, 0.02480995372926002]


# Running

In [95]:
def Bias():

    current_time = Time.time()
    duration = 1
    values = []
    while Time.time()< current_time + duration:
        values.append(np.array(sensor.read_avg_voltage()))

    values = np.array(values)
    values = [ft.mean() for ft in values.transpose()]
    sensor.set_bias(values)
    # print(values)



def record_session(T1, T2):
    global dT, q1, q2

    print(f"Starting T1 = {T1} and T2 = {T2}")
    start_time = dt.datetime.today().timestamp()
    t = 0
    l1 = 0.275
    l2 = 0.24

    logs = list()

    t_mapping = 1

    command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
    command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
    

    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    vel1 = 0
    vel2 = 0
    t1 = 0
    t2 = 0
    
    #M1
    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()

    if id == 1:
        q1 = p
        vel1 = v
        t1 = t/t_mapping
    
    elif id == 2:
        q2 = p
        vel2 = v
        t2 = t/t_mapping

    #M2
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    if id == 1:
        q1 = p
        vel1 = v
        t1 = t/t_mapping
    
    elif id == 2:
        q2 = p
        vel2 = v
        t2 = t/t_mapping
    
    
    sleep(2)


    x = l1*np.sin(q1) + l2*np.cos(q2)
    y = l1*np.cos(q1) - l2*np.sin(q2)

    time = 0
    record_duration = 5

    print("Start Controling")
    sensor.start_task()
    Bias()
    while time < record_duration:
        start_time = dt.datetime.today().timestamp()

        log = list()
        log.append(time)


        # x_old = x
        # y_old = y
        
        log.append(T1)
        log.append(T2)

        command1 = bytes(pack_cmd(0,0,T1*t_mapping,0,0)) #(pos,vel,torque,kp,kd)
        command2 = bytes(pack_cmd(0,0,-T2*t_mapping,0,0)) #(pos,vel,torque,kp,kd)

        #M1
        serD.write(bytes([0x01])+command1)
        [id, p, v, t] = receive()

        if id == 1:
            q1 = p
            vel1 = v
            t1 = t/t_mapping
        
        elif id == 2:
            q2 = p
            vel2 = v
            t2 = t/t_mapping

        #M2
        serD.write(bytes([0x02])+command2)
        [id, p, v, t] = receive()

        if id == 1:
            q1 = p
            vel1 = v
            t1 = t/t_mapping
        
        elif id == 2:
            q2 = p
            vel2 = v
            t2 = t/t_mapping


        log.append(t1)
        log.append(t2)

        log.append(q1)
        log.append(q2)

        # print("q1: ",q1, "\tq2: ", q2)

        x = l1*np.sin(q1) + l2*np.cos(q2)
        y = l1*np.cos(q1) - l2*np.sin(q2)

        log.append(x)
        log.append(y)

        # print("x: ",x, "\ty: ", y)


        x_dot = l1*np.cos(q1)*vel1 - l2*np.sin(q2)*vel2
        y_dot = -l1*np.sin(q1)*vel1 - l2*np.cos(q2)*vel2
        

        log.append(x_dot)
        log.append(y_dot)

        #Estimate CF
    
        Fx = (T1*l2*np.cos(q2)-T2*l1*np.sin(q1))/(l1*l2*np.cos(q1-q2))
        Fy = (-T1*l2*np.sin(q2)-T2*l1*np.cos(q1))/(l1*l2*np.cos(q1-q2)) 

        log.append(Fx)
        log.append(Fy)


        #Sensor
        reading = sensor.read_ft()
    

        log.append(-reading[0])
        log.append(-reading[1])

        logs.append(log)
        
        # print("Progress", f'{round(time/record_duration*100,3)}%')
        dT = dt.datetime.today().timestamp()-start_time
        time+= dT


    sensor.stop_task()

    command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
    command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)

    #M1
    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()

    if id == 1:
        q1 = p
        vel1 = v
        t1 = t/t_mapping

    elif id == 2:
        q2 = p
        vel2 = v
        t2 = t/t_mapping

    #M2
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()

    if id == 1:
        q1 = p
        vel1 = v
        t1 = t/t_mapping

    elif id == 2:
        q2 = p
        vel2 = v
        t2 = t/t_mapping

    

    print(f"finishing T1 = {T1} and T2 = {T2}")
    sleep(1)
    return logs


In [96]:
Folder = "X25"
torque_mag = 2
torque_settings = [[0, torque_mag], [torque_mag, 0], [torque_mag, torque_mag],
                   [0, -torque_mag], [-torque_mag, 0], [-torque_mag, -torque_mag]]




print("Set Zero Stepper")

FB = serS.read_all()
while True:    
    FB = serS.readline().decode('utf-8').strip()
    if FB:
        if int(FB) == 4:
            serS.write(b'2')
            break
    else:
        serS.write(b'1')

print("Ending Zero Stepper")

end = 0
count = 0
step_distance = 1.5 #cm
step_time_stepper = step_distance*1.4
total_dis = 9.623
total_count = int(total_dis//step_distance)
Y_Pos_Record = [int(100+i*step_distance*10) for i in range(total_count+1)]

while count < total_count+1:

    

    command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
    command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
    #M1
    serD.write(bytes([0x01])+command1)
    [id, p, v, t] = receive()
    #M2
    serD.write(bytes([0x02])+command2)
    [id, p, v, t] = receive()     
    sleep(2)

    for session in torque_settings:
        logs = record_session(session[0], session[1])
        log_COLUMNS = ["Time", "T1", "T2", "t1", "t2", "q1", "q2", "x", "y", "x_dot", "y_dot", "Fx", "Fy", "Fx_Mea", "Fy_Mea"]

        log_arrays = np.array([np.array(item) for item in logs])

        log_arrays_transpose = log_arrays.transpose()
        df_log = pd.DataFrame([])
        for i in range(len(log_COLUMNS)):
            df_log[log_COLUMNS[i]] = log_arrays_transpose[i]

        # print(df_log)
        df_log.to_csv(f"Record_Data/T{torque_mag}/{Folder}/Y{Y_Pos_Record[count]}_T{session[0]}{session[1]}_Static.csv")

    count+=1
    if count > total_count:
        break
    
    serS.write(b'3')
    sleep(step_time_stepper)
    serS.write(b'2')
    
    FB = serS.read_all()   
    FB = serS.readline().decode('utf-8').strip()
    if FB:
        if int(FB) == 5:
            serS.write(b'1')
            sleep(0.5)
            serS.write(b'2')
            break
    


##---Exit Motor---##
#M1
serD.write(bytes([0x01])+EXIT)
receive()

#M2
serD.write(bytes([0x02])+EXIT)
receive()

print("End all session")

Set Zero Stepper
Ending Zero Stepper
Starting T1 = 0 and T2 = 2
Start Controling
finishing T1 = 0 and T2 = 2
Starting T1 = 2 and T2 = 0
Start Controling
finishing T1 = 2 and T2 = 0
Starting T1 = 2 and T2 = 2
Start Controling
finishing T1 = 2 and T2 = 2
Starting T1 = 0 and T2 = -2
Start Controling
finishing T1 = 0 and T2 = -2
Starting T1 = -2 and T2 = 0
Start Controling
finishing T1 = -2 and T2 = 0
Starting T1 = -2 and T2 = -2
Start Controling
finishing T1 = -2 and T2 = -2
Starting T1 = 0 and T2 = 2
Start Controling
finishing T1 = 0 and T2 = 2
Starting T1 = 2 and T2 = 0
Start Controling
finishing T1 = 2 and T2 = 0
Starting T1 = 2 and T2 = 2
Start Controling
finishing T1 = 2 and T2 = 2
Starting T1 = 0 and T2 = -2
Start Controling
finishing T1 = 0 and T2 = -2
Starting T1 = -2 and T2 = 0
Start Controling
finishing T1 = -2 and T2 = 0
Starting T1 = -2 and T2 = -2
Start Controling
finishing T1 = -2 and T2 = -2
Starting T1 = 0 and T2 = 2
Start Controling
finishing T1 = 0 and T2 = 2
Starting T1

UnboundLocalError: local variable 'vel1' referenced before assignment

# Emergency Exit

In [75]:
serS.write(b'2')

command1 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)
command2 = bytes(pack_cmd(0,0,0,0,0)) #(pos,vel,torque,kp,kd)

#M1
serD.write(bytes([0x01])+command1)
[id, p, v, t] = receive()
#M2
serD.write(bytes([0x02])+command2)
[id, p, v, t] = receive()


serD.write(bytes([0x01])+EXIT)
receive()

#M2
serD.write(bytes([0x02])+EXIT)
receive()

serD.close()
serS.close()

sensor.stop_task()
